# Data Cleaning

## Data Dictionary

The raw data contains the following data per station per reading:

* Id - String - API Resource Id
* Name - String - The common name of the station
* PlaceType - String ?
* TerminalName - String - ?
* NbBikes - Integer - The number of available bikes
* NbDocks - Integer - The total number of docking spaces
* NbEmptyDocks - Integer - The number of available empty docking spaces
* Timestamp - DateTime - The moment this reading was captured
* InstallDate - DateTime - Date when the station was installed
* RemovalDate - DateTime - Date when the station was removed
* LastUpdated - DateTime - ?
* Installed - Boolean - If the station is installed or not
* Locked - Boolean - If the station is locked or not
* Temporary - Boolean - If the station is temporary or not (TfL adds temporary stations to cope with demand.)
* Latitude - Float - Latitude Coordinate
* Longitude - Float - Longitude Coordinate

The following variables will be derived from the raw data.

* NbUnusableDocks - Integer - The number of non-working docking spaces. Computed with NbUnusableDocks = NbDocks - (NbBikes + NbEmptyDocks)

## Set up

### Imports

In [286]:
import logging
import itertools
import json
import os
import pandas as pd
import pickle

from datetime import datetime
from os import listdir
from os.path import isfile, join
from src.data.parse_dataset import parse_dir, parse_json_files, get_file_list

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Parse Raw Data 

### Define the Parsing Functions

In [287]:
def parse_cycles(json_obj):
    """Parses TfL's BikePoint JSON response"""

    return [parse_station(element) for element in json_obj]

def parse_station(element):
    """Parses a JSON bicycle station object to a dictionary"""

    obj = {
        'Id': element['id'],
        'Name': element['commonName'],
        'Latitude': element['lat'],
        'Longitude': element['lon'],
        'PlaceType': element['placeType'],
    }

    for p in element['additionalProperties']:
        obj[p['key']] = p['value']

        if 'timestamp' not in obj:
            obj['Timestamp'] = p['modified']
        elif obj['Timestamp'] != p['modified']:
            raise ValueError('The properties\' timestamps for station %s do not match: %s != %s' % (
            obj['id'], obj['Timestamp'], p['modified']))

    return obj

In [288]:
def bike_file_date_fn(file_name):
    """Gets the file's date"""

    return datetime.strptime(os.path.basename(file_name), 'BIKE-%Y-%m-%d:%H:%M:%S.json')

def create_between_dates_filter(file_date_fn, date_start, date_end):
    def filter_fn(file_name):
        file_date = file_date_fn(file_name)
        return file_date >= date_start and file_date <= date_end
    
    return filter_fn

### Quick Data View

#### Load Single Day Data

In [289]:
filter_fn = create_between_dates_filter(bike_file_date_fn, 
                                       datetime(2016, 5, 16, 7, 0, 0),
                                       datetime(2016, 5, 16, 23, 59, 59))

records = parse_dir('/home/jfconavarrete/Documents/Work/Dissertation/spts-uoe/data/raw', 
                    parse_cycles, sort_fn=bike_file_date_fn, filter_fn=filter_fn)

# records is a list of lists of dicts
df = pd.DataFrame(list(itertools.chain.from_iterable(records))) 

####  All Station View

In [290]:
df.head()

,Id,InstallDate,Installed,Latitude,Locked,Longitude,Name,NbBikes,NbDocks,NbEmptyDocks,PlaceType,RemovalDate,Temporary,TerminalName,Timestamp
0,BikePoints_1,1278947280000,true,51.529163,false,-0.109970,"River Street , Clerkenwell",11,19,7,BikePoint,,false,001023,2016-05-16T06:26:24.037
1,BikePoints_2,1278585780000,true,51.499606,false,-0.197574,"Phillimore Gardens, Kensington",12,37,25,BikePoint,,false,001018,2016-05-16T06:26:24.037
2,BikePoints_3,1278240360000,true,51.521283,false,-0.084605,"Christopher Street, Liverpool Street",6,32,26,BikePoint,,false,001012,2016-05-16T06:51:27.5
3,BikePoints_4,1278241080000,true,51.530059,false,-0.120973,"St. Chad's Street, King's Cross",14,23,9,BikePoint,,false,001013,2016-05-16T06:51:27.5
4,BikePoints_5,1278241440000,true,51.493130,false,-0.156876,"Sedding Street, Sloane Square",27,27,0,BikePoint,,false,003420,2016-05-16T06:46:27.237


####  Single Station View

In [291]:
df[df['Id'] == 'BikePoints_1'].head()

,Id,InstallDate,Installed,Latitude,Locked,Longitude,Name,NbBikes,NbDocks,NbEmptyDocks,PlaceType,RemovalDate,Temporary,TerminalName,Timestamp
0,BikePoints_1,1278947280000,true,51.529163,false,-0.10997,"River Street , Clerkenwell",11,19,7,BikePoint,,false,001023,2016-05-16T06:26:24.037
762,BikePoints_1,1278947280000,true,51.529163,false,-0.10997,"River Street , Clerkenwell",11,19,7,BikePoint,,false,001023,2016-05-16T06:26:24.037
1524,BikePoints_1,1278947280000,true,51.529163,false,-0.10997,"River Street , Clerkenwell",10,19,8,BikePoint,,false,001023,2016-05-16T07:01:29.163
2286,BikePoints_1,1278947280000,true,51.529163,false,-0.10997,"River Street , Clerkenwell",8,19,10,BikePoint,,false,001023,2016-05-16T07:11:30.433
3048,BikePoints_1,1278947280000,true,51.529163,false,-0.10997,"River Street , Clerkenwell",8,19,10,BikePoint,,false,001023,2016-05-16T07:11:30.433


#### Observations

* There are some duplicate rows <- remove duplicates
* Id, Name and Terminal name seem to be candidate keys <- remove proxy variables
* RemovalDate may contain a lot of nulls <- remove if not helpful
* Locked and Installed might be constant <- remove if not helpful
* Longitude and Latitude are geographic coordinates  <- check how this can be handled

### Build Dataset

#### Work with Chunks

Due to memory constraints we'll parse the data in chunks. In each chunk we'll remove the redundant candidate keys and also duplicate rows.

In [292]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

#### Tables

We will have two different tables, one for the stations and one for the availability readings

In [293]:
def split_data(parsed_data):
    master_df = pd.DataFrame(list(itertools.chain.from_iterable(parsed_data)))
    
    readings_df = pd.DataFrame(master_df, columns=['Id', 'Timestamp', 'NbBikes', 'NbDocks', 'NbEmptyDocks'])
    stations_df = pd.DataFrame(master_df, columns=['Id', 'Name', 'TerminalName' , 'PlaceType', 'Latitude', 
                                                   'Longitude', 'Installed', 'Temporary', 'Locked',
                                                   'RemovalDate', 'InstallDate'])
    
    return (readings_df, stations_df)

#### Build the Dataset

In [294]:
# get the files to parse
five_weekdays_filter = create_between_dates_filter(bike_file_date_fn, 
                                                   datetime(2016, 5, 16, 11, 0, 0), 
                                                   datetime(2016, 5, 20, 23, 59, 59))
files = get_file_list('data/raw', filter_fn=five_weekdays_filter, sort_fn=bike_file_date_fn)

# process the files in chunks
files_batches = chunker(files, 100)

In [295]:
# start with an empty dataset
readings_dataset = pd.DataFrame()
stations_dataset = pd.DataFrame()

# append each chunk to the datasets while removing duplicates
for batch in files_batches:
    parsed_data = parse_json_files(batch, parse_cycles)
    
    # split the data into two station data and readings data
    readings_df, stations_df = split_data(parsed_data)
    
    # append the datasets
    readings_dataset = pd.concat([readings_dataset, readings_df])
    stations_dataset = pd.concat([stations_dataset, stations_df])
    
    # remove duplicated rows
    readings_dataset.drop_duplicates(inplace=True)
    stations_dataset.drop_duplicates(inplace=True)

## Technically Correct Data

The data is set to be technically correct if it:

1. can be directly recognized as belonging to a certain variable
2. is stored in a data type that represents the value domain of the real-world variable.

In [296]:
# convert columns to their appropriate datatypes
stations_dataset['InstallDate'] = pd.to_numeric(stations_dataset['InstallDate'], errors='raise')
stations_dataset['Installed'] = stations_dataset['Installed'].astype('bool_')
stations_dataset['Temporary'] = stations_dataset['Temporary'].astype('bool_')
stations_dataset['Locked'] = stations_dataset['Locked'].astype('bool_')

readings_dataset['NbBikes'] = readings_dataset['NbBikes'].astype('uint16')
readings_dataset['NbDocks'] = readings_dataset['NbDocks'].astype('uint16')
readings_dataset['NbEmptyDocks'] = readings_dataset['NbEmptyDocks'].astype('uint16')

# convert string timestamp to datetime
stations_dataset['InstallDate'] = pd.to_datetime(stations_dataset['InstallDate'], unit='ms')

readings_dataset['Timestamp'] =  pd.to_datetime(readings_dataset['Timestamp'], format='%Y-%m-%dT%H:%M:%S.%f', errors='raise')

In [297]:
# sort the datasets
stations_dataset.sort_values(by=['Id'], ascending=True, inplace=True)

readings_dataset.sort_values(by=['Timestamp'], ascending=True, inplace=True)

## Derive Data

In [298]:
readings_dataset['NbUnusableDocks'] = readings_dataset['NbDocks'] - (readings_dataset['NbBikes'] + readings_dataset['NbEmptyDocks'])

## Consistent Data

### Stations Data

#### Overview

In [299]:
stations_dataset.shape

(766, 11)

In [300]:
stations_dataset.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 94
Data columns (total 11 columns):
Id              766 non-null object
Name            766 non-null object
TerminalName    766 non-null object
PlaceType       766 non-null object
Latitude        766 non-null float64
Longitude       766 non-null float64
Installed       766 non-null bool
Temporary       766 non-null bool
Locked          766 non-null bool
RemovalDate     766 non-null object
InstallDate     677 non-null datetime64[ns]
dtypes: bool(3), datetime64[ns](1), float64(2), object(5)
memory usage: 418.1 KB


In [301]:
stations_dataset.head()

,Id,Name,TerminalName,PlaceType,Latitude,Longitude,Installed,Temporary,Locked,RemovalDate,InstallDate
0,BikePoints_1,"River Street , Clerkenwell",001023,BikePoint,51.529163,-0.109970,True,True,True,,2010-07-12 15:08:00
9,BikePoints_10,"Park Street, Bankside",001024,BikePoint,51.505974,-0.092754,True,True,True,,2010-07-04 11:21:00
95,BikePoints_100,"Albert Embankment, Vauxhall",001059,BikePoint,51.490435,-0.122806,True,True,True,,2010-07-14 09:31:00
96,BikePoints_101,"Queen Street 1, Bank",000999,BikePoint,51.511553,-0.092940,True,True,True,,2010-07-14 10:18:00
97,BikePoints_102,"Jewry Street, Aldgate",001045,BikePoint,51.513406,-0.076793,True,True,True,,2010-07-14 10:21:00


In [302]:
stations_dataset.describe()

,Latitude,Longitude
count,766.000000,766.000000
mean,51.438694,-0.129420
std,1.861092,0.055776
min,0.000000,-0.236769
25%,51.493189,-0.173840
50%,51.509265,-0.132289
75%,51.520686,-0.092926
max,51.549369,0.000000


In [303]:
stations_dataset.apply(lambda x:x.nunique())

Id              765
Name            766
TerminalName    765
PlaceType         1
Latitude        762
Longitude       761
Installed         1
Temporary         1
Locked            1
RemovalDate       4
InstallDate     676
dtype: int64

#### Observations:
* The minimum latitude and the maximum longitude are 0
* Some stations do not have latitude nor longitude
* Id, TerminalName and Name have different unique values
* Placetype, Installed, Temporary and Locked appear to be constant
* Some stations do not have an install date
* Some Stations have a removal date (very sparse)

#### Remove Duplicate Stations

In [304]:
def find_name_changes(df):
    names_per_id_count = df.groupby('Id')['Name'].count()
    ids_with_several_names_idx = names_per_id_count[names_per_id_count != 1].index.values
    return df[df['Id'].isin(ids_with_several_names_idx)]

ids_with_several_names_df = find_name_changes(stations_dataset)
ids_with_several_names_df

,Id,Name,TerminalName,PlaceType,Latitude,Longitude,Installed,Temporary,Locked,RemovalDate,InstallDate
763,BikePoints_818,"One Tower Bridge, Southwark",300249,BikePoint,51.503127,-0.078655,True,True,True,,NaT
5347,BikePoints_818,"One Tower Bridge, Bermondsey",300249,BikePoint,51.503127,-0.078655,True,True,True,,NaT


Given these records have the same location, TerminalName and Id but different Name, we'll assume the station changed name and remove the first entries.

In [305]:
is_duplicated_id = ids_with_several_names_df.duplicated(['Id'], keep='last')
duplicated_id_idx = is_duplicated_id[is_duplicated_id == True].index

stations_dataset.drop(duplicated_id_idx, inplace=True)

In [308]:
duplicated_id_idx

Int64Index([], dtype='int64')

In [306]:
assert len(find_name_changes(stations_dataset)) == 0

AssertionError: 

## Data Summary

In [ ]:
pd.concat([a, pd.DataFrame()])

### Import into Pandas

In [ ]:
dataset = pd.DataFrame(list(itertools.chain.from_iterable(records)))

dataset.shape

In [ ]:

dataset.head()

In [ ]:

nuniques = dataset.apply(lambda x:x.nunique())
constant_cols = nuniques[nuniques == 1].index
print 'Constant columns: %s' % constant_cols
dataset = dataset.drop(constant_cols, axis=1)

### Convert to Appropriate DataTypes

### Derive Variables

### Data Description

In [ ]:
dataset.info(memory_usage='deep')

## Consistent Correct Data

In [ ]:
dataset.describe()

### Missing Values



### Outliers

### Errors

### Consistency

## Exploratory Data Analysis

### Visual Representation

### Examine Variable Relationships

### Analyze Variable Over Time

## Conclusions